In [39]:
from tensorflow.keras.models import Model
import tensorflow as tf
from tensorflow import keras
from keras.layers import Dense,Conv2D,Flatten,Dropout,MaxPooling2D,LeakyReLU,BatchNormalization
from keras import Sequential
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.layers import SpatialDropout2D
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.regularizers import l2
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras import layers, models, initializers
from tensorflow.keras.optimizers import AdamW

In [47]:
early_stop = EarlyStopping(
    monitor='val_loss',      
    patience=5,              
    restore_best_weights=True
)

In [100]:
train_ds = tf.keras.preprocessing.image_dataset_from_directory(
    "/kaggle/input/aiderdata/AIDER",
    validation_split=0.15,
    subset="training",
    seed=123,
    image_size=(256, 256),
    batch_size=64
)

val_ds = tf.keras.preprocessing.image_dataset_from_directory(
    "/kaggle/input/aiderdata/AIDER",
    validation_split=0.15,
    subset="validation",
    seed=123,
    image_size=(256, 256),
    batch_size=64
)

Found 6433 files belonging to 5 classes.
Using 5469 files for training.
Found 6433 files belonging to 5 classes.
Using 964 files for validation.


In [101]:
from tensorflow.keras import layers
data_augmentation = tf.keras.Sequential([
    layers.RandomFlip("horizontal"),         
    layers.RandomRotation(0.25),             
    layers.RandomTranslation(0.1, 0.1),
    layers.RandomContrast(0.1)
])

In [102]:
normalization_layer = layers.Rescaling(1./255)
AUTOTUNE = tf.data.AUTOTUNE
train_ds = train_ds.map(lambda x, y: (data_augmentation(normalization_layer(x)), y), num_parallel_calls=AUTOTUNE)
val_ds = val_ds.map(lambda x, y: (normalization_layer(x), y), num_parallel_calls=AUTOTUNE)
train_ds = train_ds.prefetch(buffer_size=AUTOTUNE)
val_ds = val_ds.prefetch(buffer_size=AUTOTUNE)

In [103]:
model= Sequential()

model.add(Conv2D(32,kernel_size=(3,3),padding='same',input_shape=(256,256,3)))
model.add(LeakyReLU(alpha=0.1))
model.add(MaxPooling2D(pool_size=(2,2)))

model.add(Conv2D(32,kernel_size=(3,3),padding='same'))
model.add(LeakyReLU(alpha=0.1))
model.add(MaxPooling2D(pool_size=(2,2))) 

model.add(Conv2D(32,kernel_size=(3,3),padding='same'))
model.add(LeakyReLU(alpha=0.1))
model.add(MaxPooling2D(pool_size=(2,2))) 

model.add(Conv2D(32,kernel_size=(3,3),padding='same'))
model.add(LeakyReLU(alpha=0.1))
model.add(MaxPooling2D(pool_size=(2,2))) 

model.add(Conv2D(32,kernel_size=(3,3)))
model.add(LeakyReLU(alpha=0.1))
model.add(MaxPooling2D(pool_size=(2,2)))

model.add(Flatten())

model.add(Dense(64))
model.add(LeakyReLU(alpha=0.1))

model.add(Dense(5,activation='softmax'))

In [104]:
model.summary()

Model: "sequential_29"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv2d_40 (Conv2D)              │ (None, 256, 256, 32)   │           896 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ leaky_re_lu_54 (LeakyReLU)      │ (None, 256, 256, 32)   │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_39 (MaxPooling2D) │ (None, 128, 128, 32)   │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_41 (Conv2D)              │ (None, 128, 128, 32)   │         9,248 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ leaky_re_lu_55 (LeakyReLU)      │ (None, 128, 128, 32)   │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_40 (MaxPooling2D) │ (None, 64, 64, 32)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_42 (Conv2D)              │ (None, 64, 64, 32)     │         9,248 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ leaky_re_lu_56 (LeakyReLU)      │ (None, 64, 64, 32)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_41 (MaxPooling2D) │ (None, 32, 32, 32)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_43 (Conv2D)              │ (None, 32, 32, 32)     │         9,248 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ leaky_re_lu_57 (LeakyReLU)      │ (None, 32, 32, 32)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_42 (MaxPooling2D) │ (None, 16, 16, 32)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_44 (Conv2D)              │ (None, 14, 14, 32)     │         9,248 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ leaky_re_lu_58 (LeakyReLU)      │ (None, 14, 14, 32)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_43 (MaxPooling2D) │ (None, 7, 7, 32)       │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten_14 (Flatten)            │ (None, 1568)           │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_28 (Dense)                │ (None, 64)             │       100,416 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ leaky_re_lu_59 (LeakyReLU)      │ (None, 64)             │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_29 (Dense)                │ (None, 5)              │           325 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 138,629 (541.52 KB)

 Trainable params: 138,629 (541.52 KB)

 Non-trainable params: 0 (0.00 B)

In [105]:
model.compile(optimizer='adam',loss='sparse_categorical_crossentropy',metrics=['accuracy'])
history=model.fit(train_ds,epochs=30,validation_data=val_ds,callbacks=[early_stop])

Epoch 1/30
86/86 ━━━━━━━━━━━━━━━━━━━━ 46s 461ms/step - accuracy: 0.6418 - loss: 1.1547 - val_accuracy: 0.6981 - val_loss: 0.8375
Epoch 2/30
86/86 ━━━━━━━━━━━━━━━━━━━━ 39s 434ms/step - accuracy: 0.6886 - loss: 0.8431 - val_accuracy: 0.7116 - val_loss: 0.7765
Epoch 3/30
86/86 ━━━━━━━━━━━━━━━━━━━━ 39s 444ms/step - accuracy: 0.7202 - loss: 0.7771 - val_accuracy: 0.7127 - val_loss: 0.7792
Epoch 4/30
86/86 ━━━━━━━━━━━━━━━━━━━━ 40s 446ms/step - accuracy: 0.7389 - loss: 0.7119 - val_accuracy: 0.7251 - val_loss: 0.7127
Epoch 5/30
86/86 ━━━━━━━━━━━━━━━━━━━━ 39s 435ms/step - accuracy: 0.7368 - loss: 0.6856 - val_accuracy: 0.7334 - val_loss: 0.7119
Epoch 6/30
86/86 ━━━━━━━━━━━━━━━━━━━━ 39s 432ms/step - accuracy: 0.7570 - loss: 0.6368 - val_accuracy: 0.7407 - val_loss: 0.7741
Epoch 7/30
86/86 ━━━━━━━━━━━━━━━━━━━━ 39s 435ms/step - accuracy: 0.7781 - loss: 0.5916 - val_accuracy: 0.7459 - val_loss: 0.6774
Epoch 8/30
86/86 ━━━━━━━━━━━━━━━━━━━━ 39s 443ms/step - accuracy: 0.7848 - loss: 0.5625 - val_accu